In [1]:
import pandas as pd
import numpy as np

In [2]:
#!pip install pyarrow


In [3]:
#pip install fastparquet

In [4]:
import pandas as pd
import pyarrow.parquet as pq  # If using pyarrow

import pandas as pd
import fastparquet as fp  # If using fastparquet

ModuleNotFoundError: No module named 'fastparquet'

In [ ]:
rides_jan = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - JAN/yellow_tripdata_2023-01.parquet')
rides_feb = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - FEB/yellow_tripdata_2023-02.parquet')
rides_mar = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - MAR/yellow_tripdata_2023-03.parquet')
rides_apr = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - APR/yellow_tripdata_2023-04.parquet')
rides_may = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - MAY/yellow_tripdata_2023-05.parquet')
rides_jun = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - JUN/yellow_tripdata_2023-06.parquet')


In [ ]:
print("January month data shape:", rides_jan.shape)
print("February month data shape:", rides_feb.shape)
print("March month data shape:", rides_mar.shape)
print("April month data shape:", rides_apr.shape)
print("May month data shape:", rides_may.shape)
print("June month data shape:", rides_jun.shape)

### Preprocess for January - (will do this for every month and combine during visuallization(EDA))

In [ ]:
rides_jan.head()

In [ ]:
rides_jan.describe()

In [ ]:
rides_jan.info()


* Despite there being 19 columns the colums i will be focusing on will be

In [ ]:
rides_jan = rides_jan [['tpep_pickup_datetime', 'PULocationID' ]]

In [ ]:
rides_jan.rename(columns={'tpep_pickup_datetime': 'pickup_datetime',
                      'PULocationID': 'pickup_location_id'}, inplace=True)

In [ ]:
rides_jan.head()

In [ ]:
#isolated the data that falls between 2023-01-01 and 2023-02-01'
rides_jan = rides_jan[rides_jan.pickup_datetime >= '2023-01-01']
rides_jan = rides_jan[rides_jan.pickup_datetime < '2023-02-01']
rides_jan ['pickup_datetime'].describe()

In [ ]:
rides_jan.shape

In [ ]:
#Rounded data to the hour and will work with data in hourly intervals
rides_jan['pickup_hour'] = rides_jan['pickup_datetime'].dt.floor('H')

In [ ]:
#going to group location ID and and pick up hour - (.size will give me the count of ride per hour - .reset_index goives me the data in ascending order)
agg_rides_jan = rides_jan.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides_jan.rename(columns={0: 'number_of_rides'}, inplace=True) #inplace=True is permenant
agg_rides_jan

In [ ]:
agg_rides_jan.shape # i want make surei  have a complete time series - the data above appears to have eliminated rows that have no rides. What has occiures is 'pcikup hours' and 'location id' that had no ride were eliminiate from the data
#there is no row - I want to add rows that had no rows

In [ ]:
#this functions purpose is to fill in the data that was not present in the original data because there where no rides - time series data requires ther to be data in every time slot regardless of whether it existed or not
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame: #take in the aggressed dataset that i presievely made and create an output a new DF which is going to be a complete timeseries

    location_ids = agg_rides['pickup_location_id'].unique() # extract all the location ids and the full range of dates (bellow)...
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H') # ... from the first date till the last date and then building a series hour by hour
    output = pd.DataFrame()
    for location_id in tqdm(location_ids): #I did this operation per location ID .....

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'number_of_rides']] #.... so i filter the data for onlyt that location ID and then we do this operatiom

        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True) #this is the Reindex operation which basically takes in a df, (agg_rides_i), and then a full range of dates i want to have ...
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index) #..... to ensure the out put df, has as many rows(and dates ) as the full range ......
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0) #... if thats not tne case it will fill it by 0)

        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id #I then add back the location ID in this final dataframe

        output = pd.concat([output, agg_rides_i]) # the i concatinate the results with the ones i got from the previous iterations and loop location id by location id

    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'}) #then create a new column name, so this is why we need to add this rename column to inert thje index
                                                                #which is now the datetime as a column and then rename it and its the pickup_hour

    return output

In [ ]:
print(agg_rides_jan.columns)

In [ ]:
#calling the function and adding the missing slots
agg_rides_all_slots_jan = add_missing_slots(agg_rides_jan)

In [ ]:

!pip install plotly


In [ ]:
pip install plotly==5.1.0

In [ ]:
#visualise the data to detect bigs in my data processing
import plotly.express as px

In [ ]:
#visualise the data to detect bigs in my data processing - check if the graph has any breaks in it
from typing import Optional, List
import plotly.express as px

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):     #this function takes in the dataframe i made previously (rides) and the optionally takes in the locations, the ones that will be plotted
    """"
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides #once i filtered the data frame I just call the plotly dot line function and it will plot multiple lines

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="number_of_rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

In [ ]:
plot_rides(agg_rides_all_slots_jan, locations=[261])

there appear to be no breakage in my dataset and all the missing data appear to have been filled

In [ ]:
print('Number of pickup locations:', rides_jan.pickup_location_id.nunique())

### TIME SERIES TO FEATURES - January
* I will use the time-series data i have created to create usefull features - one of them will be to show rides occurring in each hourly interval

In [ ]:
#The idea behind this function is to create overlapping windows of data from a time series,
# which can be especially useful for problems like forecasting. By having a window of n_features data points predict the next data point,
# we're essentially framing a forecasting problem as a supervised learning problem, allowing traditional machine learning models to be applied on time series data.
#we trying to take partition out data in 1 hours interval across 24 hours - We took the indicies
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
    ) -> list:

        stop_position = len(data) - 1 # Defines the final index position in the data. This ensures that the sub-sequences we create don't overshoot the actual data length

        # Start the first sub-sequence at index position 0 - mid_idx nfeatures
        subseq_first_idx = 0
        subseq_mid_idx = n_features
        subseq_last_idx = n_features + 1   #This index denotes the position of our target data point.
        indices = []

        while subseq_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))    #Continues creating new sub-sequences as long as the sub-sequence's end doesn't exceed the length of the data. - then each tuple in this list provides starting, middle and ending boundary for each bundary for a subsequence

            subseq_first_idx += step_size
            subseq_mid_idx += step_size
            subseq_last_idx += step_size        #These lines progress the boundaries forward in the dataset by step_size, moving to the next overlapping window.

        return indices                           #The function ends by returning the populated list of index boundaries.

In [ ]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame,
    input_seq_len: int,
    step_size: int
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML models
    """
    assert set(ts_data.columns) == {'pickup_hour', 'number_of_rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()

    for location_id in tqdm(location_ids):

        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id,
            ['pickup_hour', 'number_of_rides']
        ]

        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []
        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['number_of_rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['number_of_rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # numpy -> pandas
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # numpy -> pandas
        targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])

        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']

In [ ]:
features, targets = transform_ts_data_into_features_and_target(
    agg_rides_all_slots_jan,
    input_seq_len=24*27*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

In [ ]:
features

In [ ]:
targets

In [ ]:
tabular_data_jan = features
tabular_data_jan['target_rides_next_hour'] = targets

tabular_data_jan.to_parquet('tabular_data_jan.parquet')

In [ ]:
tabular_data_jan

### Preprocess for Feburary - (will do this for every month and combine during visuallization(EDA))

In [ ]:
rides_feb = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - FEB/yellow_tripdata_2023-02.parquet')

In [ ]:
print("February month data shape:", rides_feb.shape)

In [ ]:
rides_feb.head()

In [ ]:
rides_feb = rides_feb [['tpep_pickup_datetime', 'PULocationID' ]]

In [ ]:
rides_feb.rename(columns={'tpep_pickup_datetime': 'pickup_datetime',
                      'PULocationID': 'pickup_location_id'}, inplace=True)

In [ ]:
rides_feb.head()

In [ ]:
#isolated the data that falls between 2023-02-01 and 2023-03-01'
rides_feb = rides_feb[rides_feb.pickup_datetime >= '2023-02-01']
rides_feb = rides_feb[rides_feb.pickup_datetime < '2023-03-01']
rides_feb ['pickup_datetime'].describe()

In [ ]:
rides_feb.shape

In [ ]:
#Rounded data to the hour and will work with data in hourly intervals
rides_feb['pickup_hour'] = rides_feb['pickup_datetime'].dt.floor('H')

In [ ]:
#going to group location ID and and pick up hour - (.size will give me the count of ride per hour - .reset_index goives me the data in ascending order)
agg_rides_feb = rides_feb.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides_feb.rename(columns={0: 'number_of_rides'}, inplace=True) #inplace=True is permenant
agg_rides_feb

In [ ]:
agg_rides_feb.shape # i want make surei  have a complete time series - the data above appears to have eliminated rows that have no rides. What has occiures is 'pcikup hours' and 'location id' that had no ride were eliminiate from the data
#there is no row - I want to add rows that had no rows

In [ ]:
#this functions purpose is to fill in the data that was not present in the original data because there where no rides - time series data requires ther to be data in every time slot regardless of whether it existed or not
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame: #take in the aggressed dataset that i presievely made and create an output a new DF which is going to be a complete timeseries

    location_ids = agg_rides['pickup_location_id'].unique() # extract all the location ids and the full range of dates (bellow)...
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H') # ... from the first date till the last date and then building a series hour by hour
    output = pd.DataFrame()
    for location_id in tqdm(location_ids): #I did this operation per location ID .....

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'number_of_rides']] #.... so i filter the data for onlyt that location ID and then we do this operatiom

        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True) #this is the Reindex operation which basically takes in a df, (agg_rides_i), and then a full range of dates i want to have ...
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index) #..... to ensure the out put df, has as many rows(and dates ) as the full range ......
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0) #... if thats not tne case it will fill it by 0)

        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id #I then add back the location ID in this final dataframe

        output = pd.concat([output, agg_rides_i]) # the i concatinate the results with the ones i got from the previous iterations and loop location id by location id

    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'}) #then create a new column name, so this is why we need to add this rename column to inert thje index
                                                                #which is now the datetime as a column and then rename it and its the pickup_hour

    return output

In [ ]:
print(agg_rides_feb.columns)

In [ ]:
#calling the function and adding the missing slots
agg_rides_all_slots_feb = add_missing_slots(agg_rides_feb)

In [ ]:
#visualise the data to detect bigs in my data processing - check if the graph has any breaks in it
from typing import Optional, List
import plotly.express as px

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):     #this function takes in the dataframe i made previously (rides) and the optionally takes in the locations, the ones that will be plotted
    """"
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides #once i filtered the data frame I just call the plotly dot line function and it will plot multiple lines

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="number_of_rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

In [ ]:
plot_rides(agg_rides_all_slots_feb, locations=[261])

In [ ]:
print('Number of pickup locations:', rides_feb.pickup_location_id.nunique())

### TIME SERIES TO FEATURES - February
* I will use the time-series data i have created to create usefull features - one of them will be to show rides occurring in each hourly interval

In [ ]:
#The idea behind this function is to create overlapping windows of data from a time series,
# which can be especially useful for problems like forecasting. By having a window of n_features data points predict the next data point,
# we're essentially framing a forecasting problem as a supervised learning problem, allowing traditional machine learning models to be applied on time series data.
#we trying to take partition out data in 1 hours interval across 24 hours - We took the indicies
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
    ) -> list:

        stop_position = len(data) - 1 # Defines the final index position in the data. This ensures that the sub-sequences we create don't overshoot the actual data length

        # Start the first sub-sequence at index position 0 - mid_idx nfeatures
        subseq_first_idx = 0
        subseq_mid_idx = n_features
        subseq_last_idx = n_features + 1   #This index denotes the position of our target data point.
        indices = []

        while subseq_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))    #Continues creating new sub-sequences as long as the sub-sequence's end doesn't exceed the length of the data. - then each tuple in this list provides starting, middle and ending boundary for each bundary for a subsequence

            subseq_first_idx += step_size
            subseq_mid_idx += step_size
            subseq_last_idx += step_size        #These lines progress the boundaries forward in the dataset by step_size, moving to the next overlapping window.

        return indices                           #The function ends by returning the populated list of index boundaries.

In [ ]:
features, targets = transform_ts_data_into_features_and_target(
    agg_rides_all_slots_feb,
    input_seq_len=24*27*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

In [ ]:
features

In [ ]:
targets

In [ ]:
tabular_data_feb = features
tabular_data_feb['target_rides_next_hour'] = targets

tabular_data_feb.to_parquet('tabular_data_feb.parquet')

In [ ]:
tabular_data_feb

### Preprocess for March - (will do this for every month and combine during visuallization(EDA))

In [ ]:
rides_mar = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - MAR/yellow_tripdata_2023-03.parquet')

In [ ]:
print("March month data shape:", rides_mar.shape)

In [ ]:
rides_mar.head()

In [ ]:
rides_mar = rides_mar [['tpep_pickup_datetime', 'PULocationID' ]]

In [ ]:
rides_mar.rename(columns={'tpep_pickup_datetime': 'pickup_datetime',
                      'PULocationID': 'pickup_location_id'}, inplace=True)

In [ ]:
rides_mar.head()


In [ ]:
#isolated the data that falls between 2023-03-01 and 2023-04-01'
rides_mar = rides_mar[rides_mar.pickup_datetime >= '2023-03-01']
rides_mar = rides_mar[rides_mar.pickup_datetime < '2023-04-01']
rides_mar ['pickup_datetime'].describe()

In [ ]:
#Rounded data to the hour and will work with data in hourly intervals
rides_mar['pickup_hour'] = rides_mar['pickup_datetime'].dt.floor('H')

In [ ]:
#going to group location ID and and pick up hour - (.size will give me the count of ride per hour - .reset_index goives me the data in ascending order)
agg_rides_mar = rides_mar.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides_mar.rename(columns={0: 'number_of_rides'}, inplace=True) #inplace=True is permenant
agg_rides_mar

In [ ]:
agg_rides_mar.shape


In [ ]:
#this functions purpose is to fill in the data that was not present in the original data because there where no rides - time series data requires ther to be data in every time slot regardless of whether it existed or not
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame: #take in the aggressed dataset that i presievely made and create an output a new DF which is going to be a complete timeseries

    location_ids = agg_rides['pickup_location_id'].unique() # extract all the location ids and the full range of dates (bellow)...
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H') # ... from the first date till the last date and then building a series hour by hour
    output = pd.DataFrame()
    for location_id in tqdm(location_ids): #I did this operation per location ID .....

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'number_of_rides']] #.... so i filter the data for onlyt that location ID and then we do this operatiom

        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True) #this is the Reindex operation which basically takes in a df, (agg_rides_i), and then a full range of dates i want to have ...
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index) #..... to ensure the out put df, has as many rows(and dates ) as the full range ......
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0) #... if thats not tne case it will fill it by 0)

        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id #I then add back the location ID in this final dataframe

        output = pd.concat([output, agg_rides_i]) # the i concatinate the results with the ones i got from the previous iterations and loop location id by location id

    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'}) #then create a new column name, so this is why we need to add this rename column to inert thje index
                                                                #which is now the datetime as a column and then rename it and its the pickup_hour

    return output


In [ ]:
print(agg_rides_mar.columns)

In [ ]:
#calling the function and adding the missing slots
agg_rides_all_slots_mar = add_missing_slots(agg_rides_mar)

In [ ]:
#visualise the data to detect bigs in my data processing - check if the graph has any breaks in it
from typing import Optional, List
import plotly.express as px

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):     #this function takes in the dataframe i made previously (rides) and the optionally takes in the locations, the ones that will be plotted
    """"
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides #once i filtered the data frame I just call the plotly dot line function and it will plot multiple lines

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="number_of_rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()


In [ ]:
plot_rides(agg_rides_all_slots_mar, locations=[261])

In [ ]:
print('Number of pickup locations:', rides_mar.pickup_location_id.nunique())


### Time series - turning time series data to features


In [ ]:
#The idea behind this function is to create overlapping windows of data from a time series,
# which can be especially useful for problems like forecasting. By having a window of n_features data points predict the next data point,
# we're essentially framing a forecasting problem as a supervised learning problem, allowing traditional machine learning models to be applied on time series data.
#we trying to take partition out data in 1 hours interval across 24 hours - We took the indicies
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
    ) -> list:

        stop_position = len(data) - 1 # Defines the final index position in the data. This ensures that the sub-sequences we create don't overshoot the actual data length

        # Start the first sub-sequence at index position 0 - mid_idx nfeatures
        subseq_first_idx = 0
        subseq_mid_idx = n_features
        subseq_last_idx = n_features + 1   #This index denotes the position of our target data point.
        indices = []

        while subseq_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))    #Continues creating new sub-sequences as long as the sub-sequence's end doesn't exceed the length of the data. - then each tuple in this list provides starting, middle and ending boundary for each bundary for a subsequence

            subseq_first_idx += step_size
            subseq_mid_idx += step_size
            subseq_last_idx += step_size        #These lines progress the boundaries forward in the dataset by step_size, moving to the next overlapping window.

        return indices                           #The function ends by returning the populated list of index boundaries.


In [ ]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame,
    input_seq_len: int,
    step_size: int
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML models
    """
    assert set(ts_data.columns) == {'pickup_hour', 'number_of_rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()

    for location_id in tqdm(location_ids):

        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id,
            ['pickup_hour', 'number_of_rides']
        ]

        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []
        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['number_of_rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['number_of_rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # numpy -> pandas
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # numpy -> pandas
        targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])

        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']


In [ ]:
features, targets = transform_ts_data_into_features_and_target(
    agg_rides_all_slots_mar,
    input_seq_len=24*27*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

In [ ]:
features


In [ ]:
targets


In [ ]:
tabular_data_mar = features
tabular_data_mar['target_rides_next_hour'] = targets

tabular_data_mar.to_parquet('tabular_data_mar.parquet')

In [ ]:
tabular_data_mar


### Preprocess for April- (will do this for every month and combine during visuallization(EDA))

In [ ]:
rides_apr = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - APR/yellow_tripdata_2023-04.parquet')

In [ ]:
print("April month data shape:", rides_apr.shape)

In [ ]:
rides_apr.head()

In [ ]:
#isolated the data that falls between 2023-01-01 and 2023-02-01'
# rides_feb = rides_feb[rides_feb.pickup_datetime >= '2023-02-01'] rides_feb = rides_feb[rides_feb.pickup_datetime < '2023-03-01'] rides_feb ['pickup_datetime'].describe()

In [ ]:
rides_apr = rides_apr [['tpep_pickup_datetime', 'PULocationID' ]]

In [ ]:
rides_apr.rename(columns={'tpep_pickup_datetime': 'pickup_datetime',
                      'PULocationID': 'pickup_location_id'}, inplace=True)

In [ ]:
rides_apr.head()

In [ ]:
#isolated the data that falls between 2023-04-01 and 2023-05-01'
rides_apr = rides_apr[rides_apr.pickup_datetime >= '2023-04-01']
rides_apr = rides_apr[rides_apr.pickup_datetime < '2023-05-01']
rides_apr ['pickup_datetime'].describe()

In [ ]:
#Rounded data to the hour and will work with data in hourly intervals
rides_apr['pickup_hour'] = rides_apr['pickup_datetime'].dt.floor('H')

In [ ]:

#going to group location ID and and pick up hour - (.size will give me the count of ride per hour - .reset_index goives me the data in ascending order)
agg_rides_apr = rides_apr.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides_apr.rename(columns={0: 'number_of_rides'}, inplace=True) #inplace=True is permenant
agg_rides_apr

In [ ]:
agg_rides_apr.shape

In [ ]:
#this functions purpose is to fill in the data that was not present in the original data because there where no rides - time series data requires ther to be data in every time slot regardless of whether it existed or not
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame: #take in the aggressed dataset that i presievely made and create an output a new DF which is going to be a complete timeseries

    location_ids = agg_rides['pickup_location_id'].unique() # extract all the location ids and the full range of dates (bellow)...
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H') # ... from the first date till the last date and then building a series hour by hour
    output = pd.DataFrame()
    for location_id in tqdm(location_ids): #I did this operation per location ID .....

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'number_of_rides']] #.... so i filter the data for onlyt that location ID and then we do this operatiom

        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True) #this is the Reindex operation which basically takes in a df, (agg_rides_i), and then a full range of dates i want to have ...
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index) #..... to ensure the out put df, has as many rows(and dates ) as the full range ......
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0) #... if thats not tne case it will fill it by 0)

        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id #I then add back the location ID in this final dataframe

        output = pd.concat([output, agg_rides_i]) # the i concatinate the results with the ones i got from the previous iterations and loop location id by location id

    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'}) #then create a new column name, so this is why we need to add this rename column to inert thje index
                                                                #which is now the datetime as a column and then rename it and its the pickup_hour

    return output


In [ ]:
print(agg_rides_apr.columns)

In [ ]:
#calling the function and adding the missing slots
agg_rides_all_slots_apr = add_missing_slots(agg_rides_apr)

In [ ]:
#visualise the data to detect bigs in my data processing - check if the graph has any breaks in it
from typing import Optional, List
import plotly.express as px

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):     #this function takes in the dataframe i made previously (rides) and the optionally takes in the locations, the ones that will be plotted
    """"
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides #once i filtered the data frame I just call the plotly dot line function and it will plot multiple lines

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="number_of_rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()


In [ ]:
plot_rides(agg_rides_all_slots_apr, locations=[261])

In [ ]:
print('Number of pickup locations:', rides_apr.pickup_location_id.nunique())

### time series data to features

In [ ]:
#The idea behind this function is to create overlapping windows of data from a time series,
# which can be especially useful for problems like forecasting. By having a window of n_features data points predict the next data point,
# we're essentially framing a forecasting problem as a supervised learning problem, allowing traditional machine learning models to be applied on time series data.
#we trying to take partition out data in 1 hours interval across 24 hours - We took the indicies
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
    ) -> list:

        stop_position = len(data) - 1 # Defines the final index position in the data. This ensures that the sub-sequences we create don't overshoot the actual data length

        # Start the first sub-sequence at index position 0 - mid_idx nfeatures
        subseq_first_idx = 0
        subseq_mid_idx = n_features
        subseq_last_idx = n_features + 1   #This index denotes the position of our target data point.
        indices = []

        while subseq_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))    #Continues creating new sub-sequences as long as the sub-sequence's end doesn't exceed the length of the data. - then each tuple in this list provides starting, middle and ending boundary for each bundary for a subsequence

            subseq_first_idx += step_size
            subseq_mid_idx += step_size
            subseq_last_idx += step_size        #These lines progress the boundaries forward in the dataset by step_size, moving to the next overlapping window.

        return indices                           #The function ends by returning the populated list of index boundaries.


In [ ]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame,
    input_seq_len: int,
    step_size: int
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML models
    """
    assert set(ts_data.columns) == {'pickup_hour', 'number_of_rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()

    for location_id in tqdm(location_ids):

        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id,
            ['pickup_hour', 'number_of_rides']
        ]

        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []
        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['number_of_rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['number_of_rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # numpy -> pandas
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # numpy -> pandas
        targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])

        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']

In [ ]:
features, targets = transform_ts_data_into_features_and_target(
    agg_rides_all_slots_apr,
    input_seq_len=24*27*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')


In [ ]:
features


In [ ]:
targets


In [ ]:
tabular_data_apr = features
tabular_data_apr['target_rides_next_hour'] = targets

tabular_data_apr.to_parquet('tabular_data_apr.parquet')


In [ ]:
tabular_data_apr


### Preprocess for May - (will do this for every month and combine during visuallization(EDA))

In [ ]:
rides_may = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - MAY/yellow_tripdata_2023-05.parquet')


In [ ]:
print("April month data shape:", rides_may.shape)

In [ ]:
rides_may.head()

In [ ]:
rides_may = rides_may [['tpep_pickup_datetime', 'PULocationID' ]]

In [ ]:
rides_may.rename(columns={'tpep_pickup_datetime': 'pickup_datetime',
                      'PULocationID': 'pickup_location_id'}, inplace=True)

In [ ]:
rides_may.head()


In [ ]:
#isolated the data that falls between 2023-01-01 and 2023-02-01'
rides_may = rides_may[rides_may.pickup_datetime >= '2023-05-01']
rides_may = rides_may[rides_may.pickup_datetime < '2023-06-01']
rides_may ['pickup_datetime'].describe()

In [ ]:
#Rounded data to the hour and will work with data in hourly intervals
rides_may['pickup_hour'] = rides_may['pickup_datetime'].dt.floor('H')

In [ ]:
#going to group location ID and and pick up hour - (.size will give me the count of ride per hour - .reset_index goives me the data in ascending order)
agg_rides_may = rides_may.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides_may.rename(columns={0: 'number_of_rides'}, inplace=True) #inplace=True is permenant
agg_rides_may


agg_rides_may.shape



In [ ]:
#this functions purpose is to fill in the data that was not present in the original data because there where no rides - time series data requires ther to be data in every time slot regardless of whether it existed or not
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame: #take in the aggressed dataset that i presievely made and create an output a new DF which is going to be a complete timeseries

    location_ids = agg_rides['pickup_location_id'].unique() # extract all the location ids and the full range of dates (bellow)...
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H') # ... from the first date till the last date and then building a series hour by hour
    output = pd.DataFrame()
    for location_id in tqdm(location_ids): #I did this operation per location ID .....

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'number_of_rides']] #.... so i filter the data for onlyt that location ID and then we do this operatiom

        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True) #this is the Reindex operation which basically takes in a df, (agg_rides_i), and then a full range of dates i want to have ...
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index) #..... to ensure the out put df, has as many rows(and dates ) as the full range ......
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0) #... if thats not tne case it will fill it by 0)

        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id #I then add back the location ID in this final dataframe

        output = pd.concat([output, agg_rides_i]) # the i concatinate the results with the ones i got from the previous iterations and loop location id by location id

    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'}) #then create a new column name, so this is why we need to add this rename column to inert thje index
                                                                #which is now the datetime as a column and then rename it and its the pickup_hour

    return output


In [ ]:
print(agg_rides_may.columns)

In [ ]:

#calling the function and adding the missing slots
agg_rides_all_slots_may = add_missing_slots(agg_rides_may)


In [ ]:
#visualise the data to detect bigs in my data processing - check if the graph has any breaks in it
from typing import Optional, List
import plotly.express as px

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):     #this function takes in the dataframe i made previously (rides) and the optionally takes in the locations, the ones that will be plotted
    """"
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides #once i filtered the data frame I just call the plotly dot line function and it will plot multiple lines

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="number_of_rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()


In [ ]:
plot_rides(agg_rides_all_slots_may, locations=[261])


In [ ]:
print('Number of pickup locations:', rides_may.pickup_location_id.nunique())


TIME SERIES - TURNING TIME SERIES DATA INTO FEATURES

In [ ]:
#The idea behind this function is to create overlapping windows of data from a time series,
# which can be especially useful for problems like forecasting. By having a window of n_features data points predict the next data point,
# we're essentially framing a forecasting problem as a supervised learning problem, allowing traditional machine learning models to be applied on time series data.
#we trying to take partition out data in 1 hours interval across 24 hours - We took the indicies
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
    ) -> list:

        stop_position = len(data) - 1 # Defines the final index position in the data. This ensures that the sub-sequences we create don't overshoot the actual data length

        # Start the first sub-sequence at index position 0 - mid_idx nfeatures
        subseq_first_idx = 0
        subseq_mid_idx = n_features
        subseq_last_idx = n_features + 1   #This index denotes the position of our target data point.
        indices = []

        while subseq_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))    #Continues creating new sub-sequences as long as the sub-sequence's end doesn't exceed the length of the data. - then each tuple in this list provides starting, middle and ending boundary for each bundary for a subsequence

            subseq_first_idx += step_size
            subseq_mid_idx += step_size
            subseq_last_idx += step_size        #These lines progress the boundaries forward in the dataset by step_size, moving to the next overlapping window.

        return indices                           #The function ends by returning the populated list of index boundaries

In [ ]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame,
    input_seq_len: int,
    step_size: int
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML models
    """
    assert set(ts_data.columns) == {'pickup_hour', 'number_of_rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()

    for location_id in tqdm(location_ids):

        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id,
            ['pickup_hour', 'number_of_rides']
        ]

        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []
        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['number_of_rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['number_of_rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # numpy -> pandas
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # numpy -> pandas
        targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])

        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']


In [ ]:
features, targets = transform_ts_data_into_features_and_target(
    agg_rides_all_slots_may,
    input_seq_len=24*27*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')


In [ ]:
features

In [ ]:
targets

In [ ]:
tabular_data_may = features
tabular_data_may['target_rides_next_hour'] = targets

tabular_data_may.to_parquet('tabular_data_may.parquet')

In [ ]:
tabular_data_may


### Preprocess for June - (will do this for every month and combine during visuallization(EDA))

In [ ]:
rides_jun = pd.read_parquet('/Users/chetanhalai/Documents/code base for projects/4) uber/FINAL TAXI PROJECT 2023/data/2023 - JUN/yellow_tripdata_2023-06.parquet')


In [ ]:
print("April month data shape:", rides_jun.shape)

In [ ]:
rides_jun.head()

In [ ]:
rides_jun = rides_jun [['tpep_pickup_datetime', 'PULocationID' ]]

In [ ]:
rides_jun.rename(columns={'tpep_pickup_datetime': 'pickup_datetime',
                      'PULocationID': 'pickup_location_id'}, inplace=True)

In [ ]:
rides_jun.head()

In [ ]:
rides_jun = rides_jun[rides_jun.pickup_datetime >= '2023-06-01']
rides_jun = rides_jun[rides_jun.pickup_datetime < '2023-07-01']
rides_jun ['pickup_datetime'].describe()

In [ ]:
#Rounded data to the hour and will work with data in hourly intervals
rides_jun['pickup_hour'] = rides_jun['pickup_datetime'].dt.floor('H')

In [ ]:
#going to group location ID and and pick up hour - (.size will give me the count of ride per hour - .reset_index goives me the data in ascending order)
agg_rides_jun = rides_jun.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides_jun.rename(columns={0: 'number_of_rides'}, inplace=True) #inplace=True is permenant
agg_rides_jun

In [ ]:
agg_rides_jun.shape

In [ ]:
#this functions purpose is to fill in the data that was not present in the original data because there where no rides - time series data requires ther to be data in every time slot regardless of whether it existed or not
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame: #take in the aggressed dataset that i presievely made and create an output a new DF which is going to be a complete timeseries

    location_ids = agg_rides['pickup_location_id'].unique() # extract all the location ids and the full range of dates (bellow)...
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H') # ... from the first date till the last date and then building a series hour by hour
    output = pd.DataFrame()
    for location_id in tqdm(location_ids): #I did this operation per location ID .....

        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour', 'number_of_rides']] #.... so i filter the data for onlyt that location ID and then we do this operatiom

        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True) #this is the Reindex operation which basically takes in a df, (agg_rides_i), and then a full range of dates i want to have ...
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index) #..... to ensure the out put df, has as many rows(and dates ) as the full range ......
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0) #... if thats not tne case it will fill it by 0)

        # add back `location_id` columns
        agg_rides_i['pickup_location_id'] = location_id #I then add back the location ID in this final dataframe

        output = pd.concat([output, agg_rides_i]) # the i concatinate the results with the ones i got from the previous iterations and loop location id by location id

    # move the purchase_day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index': 'pickup_hour'}) #then create a new column name, so this is why we need to add this rename column to inert thje index
                                                                #which is now the datetime as a column and then rename it and its the pickup_hour

    return output


In [ ]:
print(agg_rides_jun.columns)

In [ ]:
#calling the function and adding the missing slots
agg_rides_all_slots_jun = add_missing_slots(agg_rides_jun)


In [ ]:
#visualise the data to detect bigs in my data processing - check if the graph has any breaks in it
from typing import Optional, List
import plotly.express as px

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):     #this function takes in the dataframe i made previously (rides) and the optionally takes in the locations, the ones that will be plotted
    """"
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides #once i filtered the data frame I just call the plotly dot line function and it will plot multiple lines

    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="number_of_rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

In [ ]:
plot_rides(agg_rides_all_slots_jun, locations=[261])

In [ ]:
print('Number of pickup locations:', rides_jun.nunique())

### time series - turning time into features

In [ ]:
#The idea behind this function is to create overlapping windows of data from a time series,
# which can be especially useful for problems like forecasting. By having a window of n_features data points predict the next data point,
# we're essentially framing a forecasting problem as a supervised learning problem, allowing traditional machine learning models to be applied on time series data.
#we trying to take partition out data in 1 hours interval across 24 hours - We took the indicies
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
    ) -> list:

        stop_position = len(data) - 1 # Defines the final index position in the data. This ensures that the sub-sequences we create don't overshoot the actual data length

        # Start the first sub-sequence at index position 0 - mid_idx nfeatures
        subseq_first_idx = 0
        subseq_mid_idx = n_features
        subseq_last_idx = n_features + 1   #This index denotes the position of our target data point.
        indices = []

        while subseq_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))    #Continues creating new sub-sequences as long as the sub-sequence's end doesn't exceed the length of the data. - then each tuple in this list provides starting, middle and ending boundary for each bundary for a subsequence

            subseq_first_idx += step_size
            subseq_mid_idx += step_size
            subseq_last_idx += step_size        #These lines progress the boundaries forward in the dataset by step_size, moving to the next overlapping window.

        return indices                           #The function ends by returning the populated list of index boundaries.


In [ ]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame,
    input_seq_len: int,
    step_size: int
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML models
    """
    assert set(ts_data.columns) == {'pickup_hour', 'number_of_rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()

    for location_id in tqdm(location_ids):

        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id,
            ['pickup_hour', 'number_of_rides']
        ]

        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []
        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['number_of_rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['number_of_rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # numpy -> pandas
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # numpy -> pandas
        targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])

        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']

In [ ]:
features, targets = transform_ts_data_into_features_and_target(
    agg_rides_all_slots_jun,
    input_seq_len=24*27*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')


In [ ]:
features

In [ ]:
targets

In [ ]:
tabular_data_jun = features
tabular_data_jun['target_rides_next_hour'] = targets

tabular_data_jun.to_parquet('tabular_data_jun.parquet')

In [ ]:
tabular_data_jun

### Checking every months data for any defects

In [ ]:
tabular_data_jan.head(2)

In [ ]:
print(tabular_data_jan.shape)
print(tabular_data_feb.shape)
print(tabular_data_mar.shape)
print(tabular_data_apr.shape)
print(tabular_data_may.shape)
print(tabular_data_jun.shape)

In [ ]:
tabular_data_feb.head()

In [ ]:
tabular_data_mar.head()

In [ ]:
tabular_data_apr.head()

In [ ]:
tabular_data_may.head()

In [ ]:
tabular_data_jun.head()

### Concatenating all time series data - (for hopsworks feature store)


In [ ]:
ts_data_final = pd.concat([agg_rides_all_slots_jan, agg_rides_all_slots_feb, agg_rides_all_slots_mar, agg_rides_all_slots_apr, agg_rides_all_slots_may, agg_rides_all_slots_jun])

In [ ]:
ts_data_final

In [ ]:
ts_data_final.to_parquet('ts_data_final.parquet')

### CONCATENATING ALL DATAFRAMES 

In [ ]:
tabular_data_final = pd.concat([tabular_data_jan, tabular_data_feb, tabular_data_mar, tabular_data_apr, tabular_data_may, tabular_data_jun], ignore_index = True)

tabular_data_final[760:780]


In [ ]:
tabular_data_final.to_parquet('tabular_data_final.parquet')

Learning clarity 

columns = features 
varibales and dimensions 

target_rides_next_hour = dependendent varibales - target varibale 

rows = observation 

eg - first row - ( on 2023-01-28 at pick up location id 4 at the first hour there were 22rides and on the 2 hour there were 18 rides )